In [18]:
import iapws.iapws97 as wp
import pickle
from iapws import IAPWS97
import numpy as np
from scipy.integrate import *
import matplotlib.pyplot as plt
%matplotlib qt5
from dataclasses import dataclass
from typing import List

@dataclass
class Q:
    Estructuras: float
    GV: float
    Combustible: float
    SECR: float
    T_prim: float
    t: float
        
@dataclass
class Salida:
    P_Primario: List[float]
    T_Combustible: List[float]
    Masa_GV: List[float]
    T_Estructuras: List[float]
    Masa_Interior_RPV: List[float]
    Volumen_N2: List[float]
    Volumen_liquido_interior_RPV: List[float]   
    m_lok:List[float]
    m_cie:List[float]
    t:List[float]


In [19]:
#definicion de constantes
#primario
P_0=12.25 #Mpa presion nominal del reactor
T_0=IAPWS97(P=P_0,x=0.5).T
Pot_gen= 100*1e6 #Wth potencia neta generada
V_T=55.9 #m3 volumen neto total
Vol_f_0=13.31 #m3 volumen de agua hasta tope de nucleo
Vol_v_0=10 #m3volumen de vapor
M_rpv=143000 #Kg masa total RPV
M_int=53000 #Kg masa interinos
m_estr = M_rpv + M_int
mvap=770.334
mliq=29688.12
m_t=mvap+mliq
#sie
V_f_sie_0=50 #m3 volumen de agua SIE inicial
M_f_sie_0=V_f_sie_0*1000 # masa en kilogramos de el sie , el 1000 es la densidad
Vn2_0=19 #m3 volumen N2 inicial SIE de gas
T_sie_0=318 #K centigrados temperatura inicial del sie
K_sie=2.47e4 #SU factor de frccion de descarga 
Pn2_0=2 #MPa 
Dif_P_rot=0.5 #Mpa
A_sie=4.53e-3 #m2 area por donde inyecta el sie 
K_loca=2.47e4

#generador de vapor
P_sec_0=4.7 #Mpa presion inicial secundario
T_sec_0=IAPWS97(P=P_sec_0,x=0.5).T#K temperatura en el secuandario
M_f_sec_0=600 #Kg masa de fluido en el secundadrio
Ah_fg_gv_0=Pot_gen/(T_0-T_sec_0)#MW/K area liquida multiplicado por el h liquido a gas
hfg_sec = IAPWS97(P=P_sec_0,x=0.5).Hvap*1000
#combustibles
R_pas=3.8 #mm radio pastilla
R_int_vai=3.875 #mm radio interior de la vaina
R_ext_vai=4.5 #mm radio externo de la vaina
N_tot_barras=6583 #SU numero total de barras combustibles
Long_z_act=1.4 #m longitud zona activa
Cp_uo2=3e6 #j/(m3*K) capacidad calorifica UO2
Cp_vai=2e6 #j/(m3*K) capacidad calorifica vaina
m_cp_comb = 1.557375e6
#secr
P_secr_0=0.101325 #Mpa Presion en el secre, presion atmosferica
T_secr_0=IAPWS97(P=P_secr_0,x=0.5).T#K temperatura secre
ha_secr_0=2*1e6/(T_0-T_secr_0)#W/K area liquida multiplicado por el h liquido a gas

T_estr_0=600 #K
h_estr=1000 #w/kg
A_estr=86
cp_estr=500 #j/m3k
ha_comb=366300.3663

#Parametros loca
P_cont=0.1
#A_loca=1.14e-3
Qmass_in_gv=0

FLAG_LOCA=0
FLAG_SIE=0
FLAG_MASA_GEN_VAP = 0
FLAG_SECR = 0
FLAG_ESTR = 0

In [20]:
def m_loca(P,K_loca,P_cont,FLAG_LOCA):
    if(FLAG_LOCA == 0):
        return 0
    gas= wp.IAPWS97(P=P,x=0.5)
    #if(P<1.8*P_cont):
     #   m=A_loca*np.sqrt(2*gas.Vapor.rho*1e6*(P-P_cont)/K_loca)
    #  #  return m
    #if(P>=2*P_cont):
    m=A_loca*np.sqrt(1.3*P*1e6*gas.Vapor.rho*(2/(1.3+1))**(7.6666))
    return m
    #m1=A_loca*np.sqrt(2*gas.Vapor.rho*1e6*(P-P_cont)/K_loca)
    #m2=A_loca*np.sqrt(1.3*P*1e6*gas.Vapor.rho*(2/(1.3+1))**(7.6666))
    #if(m1<m2):
     #   return m1
    #return m2
                        
    #d_LOCA = 1.5*2.54/100 # m (1.5 inch)
    #A_LOCA = np.pi*(d_LOCA/2)**2 # m2 
    #rhog = wp.IAPWS97(P=P,x=1).rho
    #K_LOCA = 1.46 # Contracción y expansion abrupta (verificar!)
    #K_bloq = 1.3
    #exp_bloq = (K_bloq+1)/(K_bloq-1)

    #mflow_Bloq   = A_LOCA*(K_bloq*P*rhog*(2/(K_bloq+1))**exp_bloq)**0.5
    #mflow_noBloq = A_LOCA*(2*(P - P_cont)*rhog/K_LOCA)**0.5

    #mflow_LOCA = min(mflow_noBloq, mflow_Bloq)
    #return mflow_LOCA
#def m_loca(P,K_loca,P_cont,FLAG_LOCA):
#    if(FLAG_LOCA == 0):
#        return 0
#    gas= wp.IAPWS97(P=P,x=0.5) 
#    m1=A_loca*np.sqrt(gas.Vapor.rho*(P-P_cont)*1e6/K_loca)
#    m2=A_loca*np.sqrt(1.3*P*1e6*gas.Vapor.rho*(2/(1.3+1))**(7.6666))
#    if (m1 <= m2):
#          return m1
#    return m2
   


#def m_sie(Pn2,P,FLAG_SIE):
#    if(FLAG_SIE==0):
#        return 0
#    m_sie=A_sie*((Pn2-P)*1e6*2*IAPWS97(T=T_sie_0,P=Pn2).rho*0.5)
#    return m_sie

def m_sie(Pn2,P,FLAG_SIE):
    if(FLAG_SIE==0):
        return 0
    if(P<=0.1):
            P=0.1
    m_sie=A_sie*((Pn2-P)*1e6*2*wp.IAPWS97(T=T_sie_0,P=Pn2).rho/K_sie)**0.5
    return m_sie


def Busca_T(P):
  #Para esta expresión, la presión debe estar en megapascales
    if(P<=0.1):
            P=0.1 
    Tsat = wp._TSat_P(P)
    return Tsat

def AB(P_s):
    if(P_s<=0.1):
            P_s=0.1
    a=wp.IAPWS97(P=P_s,x=0.5)
    A=(a.Liquid.u*a.Liquid.rho*1000 - a.Vapor.u*a.Vapor.rho*1000)/(a.Liquid.rho - a.Vapor.rho)
    B=(a.Vapor.u*1000-a.Liquid.u*1000)/(1/a.Vapor.rho - 1/a.Liquid.rho)    
    return A,B

def dAB(P_s,delta):
    if(P_s<=0.1):
            P_s=0.1
    acum_1=AB(P_s+delta)
    acum_2=AB(P_s-delta)
    dA= (acum_1[0] - acum_2[0])/(delta*2)
    dB= (acum_1[1] - acum_2[1])/(delta*2)
    return dA,dB

def Q_decai(t):
    if (t<=9):
        Q=0.15*100e6*np.exp(-0.1*t)
    if (9<t<=180):
        Q=0.09192*100e6*t**(-0.181)
    if (180<t<4e6):
        Q=0.156*100e6*t**(-0.283)
    if (t>=4e6):
        Q=0
    return Q;

In [21]:
def Calcula_Qs(Info,t):
    Q_estr = np.empty_like(t)
    Q_GV = np.empty_like(t)
    Q_transf_comb = np.empty_like(t)
    Q_secr = np.empty_like(t)
    T_prim = np.empty_like(t)
    P_Primario=Info[0]
    T_Combustible=Info[1]
    Masa_GV=Info[2]
    T_Estructuras=Info[3]
    Masa_Interior_RPV=Info[4]
    Volumen_N2=Info[5]
    
    
    for i in range(0,t.size):
        T_prim[i] = Busca_T(P_Primario[i])
        Q_estr[i] = -1*(h_estr*A_estr*(T_Estructuras[i]-T_prim[i]))

        Q_GV[i] = (Ah_fg_gv_0 * (Masa_GV[i]/M_f_sec_0) *(T_prim[i] - T_sec_0))

        Q_transf_comb[i] = ha_comb*(T_Combustible[i] - T_prim[i])

        Q_secr[i] = (ha_secr_0* (T_prim[i]-T_secr_0))

    P=Q(Q_estr,Q_GV,Q_transf_comb,Q_secr,T_prim,t)
    
    return P
    

In [22]:
def model(t,z,Pot_gen):
    P = z[0]
    T_comb = z[1]
    m_T_gv = z[2]
    T_estr = z[3]
    m_T = z[4]
    Vn2= z[5]
    if(P<=0.1):
            P=0.1
    T_prim = Busca_T(P)
    T_sec = Busca_T(P_sec_0)
    F_p=AB(P)
    dF_p=dAB(P,0.01)
    SIE=wp.IAPWS97(P=Pn2_0,x=0)
    LOCA=wp.IAPWS97(P=P,x=0.5)
        
    Q_estr = -1*(h_estr*A_estr*(T_estr-T_prim))*FLAG_ESTR
    
    Q_GV = (Ah_fg_gv_0 * (m_T_gv/M_f_sec_0) *(T_prim - T_sec_0))*FLAG_GV
    
    Q_transf_comb = ha_comb*(T_comb - T_prim)
    
    Q_secr = (ha_secr_0* (T_prim-T_secr_0))*FLAG_SECR
    
    Pn2=Pn2_0*(Vn2_0/Vn2)**1.4
    #Ecuaciones diferenciales
    hota=m_sie(Pn2,P,FLAG_SIE)
    jota=m_loca(P,K_loca,P_cont,FLAG_LOCA)

    dPdt = ((Pot_gen - Q_GV - Q_estr - Q_secr + hota*(SIE.Liquid.h*1000-F_p[0]) - jota*(LOCA.Vapor.h*1000-F_p[0]))/(m_T*dF_p[0] + V_T*dF_p[1]))
    
    #Dudoso el signo de SIE.LIQUID.H+F_p[0]
    dT_combdt = (Pot_gen - Q_transf_comb)/(m_cp_comb)
    
    dm_gvdt = (-Q_GV/(hfg_sec) + Qmass_in_gv)*FLAG_MASA_GEN_VAP

    dT_estrdt = Q_estr/(m_estr*cp_estr)
    
    dm_Tdt = hota - jota
    
    dVn2dt=hota/IAPWS97(T=T_sie_0,P=Pn2).rho

    dzdt = [dPdt,dT_combdt,dm_gvdt,dT_estrdt,dm_Tdt,dVn2dt] #devuelvo las derivadas para el método
    return dzdt

In [23]:
#Condiciones iniciales
z0 = [P_0,600,600,326+273.15,m_t,Vn2_0]
t=np.linspace(0,10000,1001)
tspan=(0,7200)
Busca_T(P_0)

599.4099163118761

In [24]:
#y=solve_ivp(model,tspan,z0,method='LSODA',t_eval=t)
Busca_T(1.9)-T_secr_0

109.8315607689022

## Verificación de la ecuación de la energía

In [25]:
t = np.linspace(0,10000,10001)
z0 = [P_0,873.15,600,326+273.15,m_t,Vn2_0,]

# Calculemos el valor de la inhomogeneidad
u = np.zeros(10001)
#u=u


# change to 2.0 at time = 5.0
#u[301:451] = 2.92e6
u[100:1100]=2.913728e6


# store solution
P_Primario = np.empty_like(t)
T_Combustible = np.empty_like(t)
Masa_GV = np.empty_like(t)
T_Estructuras = np.empty_like(t)
Masa_Interior_RPV = np.empty_like(t)
Volumen_N2 = np.empty_like(t)
# record initial conditions
P_Primario[0] = z0[0]
T_Combustible[0] = z0[1]
Masa_GV[0] = z0[2]
T_Estructuras[0] = z0[3]
Masa_Interior_RPV[0] = z0[4]
Volumen_N2[0] = z0[5]

#FLAGS

FLAG_SCRAM=0
FLAG_SIE=0
FLAG_GV=0
FLAG_SECR=0
FLAG_LOCA=0
FLAG_ESTR=0
FLAG_MASA_GEN_VAP=0



# solve ODE
for i in range(1,10001):
    # span for next time step
    tspan = [t[i-1],t[i]]
    # solve for next step
    y=solve_ivp(model,tspan,z0,method='LSODA',args=(u[i],))

    # store solution for plotting
    
    P_Primario[i] = y.y[0][1]
    T_Combustible[i] = y.y[1][1]
    Masa_GV[i] = y.y[2][1]
    T_Estructuras[i] = y.y[3][1]
    Masa_Interior_RPV[i] = y.y[4][1]
    Volumen_N2[i] = y.y[5][1]
    # next initial condition
    z0 = y.y[:,1]

In [26]:
plt.figure()
plt.plot(t/60,P_Primario)
plt.title('Evolución de la presión')
plt.grid()
plt.xlabel("Tiempo[min]")
plt.ylabel("P[MPa]")

Text(0, 0.5, 'P[MPa]')

In [27]:
plt.plot(t,T_Estructuras)

In [28]:
t = np.linspace(0,20000,1001)
Q_estr = np.empty_like(t)
Q_GV = np.empty_like(t)
Q_transf_comb = np.empty_like(t)
Q_secr = np.empty_like(t)
T_prim = np.empty_like(t)
for i in range(0,1001):
    T_prim[i] = Busca_T(P_Primario[i])
    Q_estr[i] = -1*(h_estr*A_estr*(T_Estructuras[i]-T_prim[i]))
    
    Q_GV[i] = (Ah_fg_gv_0 * (Masa_GV[i]/M_f_sec_0) *(T_prim[i] - T_sec_0))
    
    Q_transf_comb[i] = ha_comb*(T_Combustible[i] - T_prim[i])
    
    Q_secr[i] = (ha_secr_0* (T_prim[i]-T_secr_0))

In [29]:
Busca_T(2.26)-273.15

218.65008662293172

In [30]:
f_f=AB(15)
f_f

(1419190.216625865, 100247966.47392315)

In [31]:
f_i=AB(12.25)
f_i

(1354717.3059358613, 83253636.06381083)

In [32]:
(m_t*(f_f[0]-f_i[0])+V_T*(f_f[1]-f_i[1]))/1000

2913728.2544228667

## Análisis del LOHS
La presión de disparo del SCRAM es de 13 MPa, el SECR es 13.7 MPa.
Para esto modifiquemos el código que hace la parte del escalón, haciendo un escalón que varíe con el tiempo.
En primer lugar, programemos la pérdida del GV.

In [33]:
t = np.linspace(0,126000,100001)
z0 = [P_0,600+273.15,600,326+273.15,m_t,Vn2_0]

# Calculemos el valor de la inhomogeneidad
u = np.zeros(100001)
u=u+100e6 #colocamos una potencia constante

# store solution
P_Primario = np.empty_like(t)
T_Combustible = np.empty_like(t)
Masa_GV = np.empty_like(t)
T_Estructuras = np.empty_like(t)
Masa_Interior_RPV = np.empty_like(t)
Volumen_N2 = np.empty_like(t)
# record initial conditions
P_Primario[0] = z0[0]
T_Combustible[0] = z0[1]
Masa_GV[0] = z0[2]
T_Estructuras[0] = z0[3]
Masa_Interior_RPV[0] = z0[4]
Volumen_N2[0] = z0[5]

#FLAGS
FLAG_SCRAM=0
FLAG_SIE=0
FLAG_GV=1
FLAG_SECR=0
FLAG_LOCA=0
FLAG_ESTR=1
FLAG_MASA_GEN_VAP=0


# solve ODE
for i in range(1,100001):
    # span for next time step
    tspan = [t[i-1],t[i]]
    # solve for next step
    if(t[i-1]>1000):
        FLAG_GV=0
    if(z0[0]>13):
        if(FLAG_SCRAM==0):
            t_scram = t[i-1]
            FLAG_SCRAM = 1
    if(FLAG_SCRAM == 1):
        u[i]=Q_decai(t[i-1]-t_scram)
    if(z0[0]>13.7):
        FLAG_SECR=1
        
    y=solve_ivp(model,tspan,z0,method='LSODA',args=(u[i],))

    # store solution for plotting
    
    P_Primario[i] = y.y[0][1]
    T_Combustible[i] = y.y[1][1]
    Masa_GV[i] = y.y[2][1]
    T_Estructuras[i] = y.y[3][1]
    Masa_Interior_RPV[i] = y.y[4][1]
    Volumen_N2[i] = y.y[5][1]
    # next initial condition
    z0 = y.y[:,1]
Salida_LOHS = (P_Primario,T_Combustible,Masa_GV,T_Estructuras,Masa_Interior_RPV,Volumen_N2)

KeyboardInterrupt: 

In [ ]:
plt.plot(t/3600,P_Primario)

In [ ]:
t = np.linspace(0,10000,100001)
Q_estr = np.empty_like(t)
Q_GV = np.empty_like(t)
Q_transf_comb = np.empty_like(t)
Q_secr = np.empty_like(t)
for i in range(0,100001):
    T_prim = Busca_T(P_Primario[i])
    Q_estr[i] = -1*(h_estr*A_estr*(T_Estructuras[i]-T_prim))
    
    Q_GV[i] = (Ah_fg_gv_0 * (Masa_GV[i]/M_f_sec_0) *(T_prim - T_sec_0))
    
    Q_transf_comb[i] = ha_comb*(T_Combustible[i] - T_prim)
    
    Q_secr[i] = (ha_secr_0* (T_prim-T_secr_0))

In [ ]:
plt.plot(t,Q_secr)

### Simulación de LOCA

In [ ]:
A_locas=1.14e-3#metros cuadrados=(30,40,50,60)
P_iny=(1.5)

V_f_sie_0s=(40,50)
Salida_LOCA_2=[]
for indice in range(0,2):
    V_f_sie_0=V_f_sie_0s[indice]
    t1 = np.linspace(0,3000,3000)
    t2 = np.linspace(3001,129000,30000)
    t = np.concatenate((t1,t2))
    z0=[P_0,T_0,M_f_sec_0,T_estr_0,m_t,Vn2_0]
    A_loca=A_locas
    
   

    u =np.zeros(t.size)
    # store solution
    P_Primario = np.empty_like(t)
    T_Combustible = np.empty_like(t)
    Masa_GV = np.empty_like(t)
    T_Estructuras = np.empty_like(t)
    Masa_Interior_RPV = np.empty_like(t)
    Volumen_N2 = np.empty_like(t)
    m_lok = np.empty_like(t)
    m_cie=np.empty_like(t)
    Volumen_interior_RPV=np.empty_like(t)
    Masa_liquido_Interior_RPV=np.empty_like(t)
    Volumen_liquido_interior_RPV=np.empty_like(t)

    P_Primario[0] = z0[0]
    T_Combustible[0] = z0[1]
    Masa_GV[0] = z0[2]
    T_Estructuras[0] = z0[3]
    Masa_Interior_RPV[0] = z0[4]
    Volumen_N2[0] = z0[5]

    #FLAGS
    FLAG_SCRAM=1
    FLAG_SIE=0
    FLAG_GV=1
    FLAG_SECR=0
    FLAG_LOCA=1
    FLAG_MASA_GEN_VAP=1
    FLAG_ESTR=1



    for i in range(1,t.size):
        # span for next time step
        tspan = [t[i-1],t[i]]
        # solve for next step
        if(z0[0]<P_iny):#Mpa
            FLAG_SIE=1
        if(z0[0]<6):
            FLAG_SECR = 1
        u[i]=Q_decai(t[i])
        if(z0[5]>=V_f_sie_0+Vn2_0):
            FLAG_SIE=0
        if(z0[0]<=0.1):
            z0[0]=0.1  
            FLAG_LOCA=0
    
        
        y=solve_ivp(model,tspan,z0,method='LSODA',args=(u[i],))
        # store solution for plotting
        
        if (y.y[0][1]<=0.1):
            y.y[0][1]=0.1
        P_Primario[i] = y.y[0][1]
        T_Combustible[i] = y.y[1][1]
        Masa_GV[i] = y.y[2][1]
        if(Masa_GV[i]<0.1):
            FLAG_GV=0
        T_Estructuras[i] = y.y[3][1]
        Masa_Interior_RPV[i] = y.y[4][1]
        Volumen_N2[i] = y.y[5][1]
        Pn2=Pn2_0*(Vn2_0/Volumen_N2[i])**1.4
        m_lok[i]=m_loca(P_Primario[i],K_loca,P_cont,FLAG_LOCA)
        m_cie[i]=m_sie(Pn2,P_Primario[i],FLAG_SIE)
        Masa_liquido_Interior_RPV[i]=(V_T-Masa_Interior_RPV[i]/IAPWS97(P=P_Primario[i],x=0.5).Vapor.rho)/(1/IAPWS97(P=P_Primario[i],x=0.5).Liquid.rho-1/IAPWS97(P=P_Primario[i],x=0.5).Vapor.rho)
        Volumen_liquido_interior_RPV[i]=Masa_liquido_Interior_RPV[i]/IAPWS97(P=P_Primario[i],x=0.5).Liquid.rho
        print(i)
       
    
        # next initial condition
        z0 = y.y[:,1]
    Salida_LOCA_2.append(Salida(P_Primario,T_Combustible,Masa_GV,T_Estructuras,Masa_Interior_RPV,Volumen_N2,Volumen_liquido_interior_RPV,m_lok,m_cie,t))
#Salida_LOCA = (P_Primario,T_Combustible,Masa_GV,T_Estructuras,Masa_Interior_RPV,Volumen_N2)

In [ ]:
fig, ax = plt.subplots()

ax.plot(t/3600,  Salida_LOCA_2[2].Volumen_liquido_interior_RPV, label=r"$y = P Primario$")
#ax.plot(t/3600, y, label=r"$x = t(h)")
ax.legend(loc=2) # upper left corner
ax.set_xlabel(r'$tiempo(h)$')
ax.set_ylabel(r'$P Primario(Mpa)$')
#ax.set_title('title');
Volumen_N2[-1]


In [ ]:
fig, ax = plt.subplots()

ax.plot(t/3600, m_lok,label=r"m-loca")
#ax.plot(t/3600, y, label=r"$x = t(h)")
ax.legend(loc=2) # upper left corner
ax.set_xlabel(r'$tiempo(h)$')
ax.set_ylabel(r'$M loca$')
#ax.set_title('title');

In [ ]:
f = open('simulacion_volumen_50_domingo_mediodia.pckl', 'wb')
pickle.dump( Salida_LOCA_2[1], f)
f.close()

In [ ]:
f = open('30volumen.pckl', 'rb')
vol30 = pickle.load(f)
f.close()